In [1]:
#  Predict Titanic Survival
#The RMS Titanic set sail on its maiden voyage in 1912, crossing the Atlantic from Southampton, England to New York City. The ship never completed the voyage, sinking to the bottom of the Atlantic Ocean after hitting an iceberg, bringing down 1,502 of 2,224 passengers onboard.
#In this project you will create a Logistic Regression model that predicts which passengers survived the sinking of the Titanic, based on features like age and class.
#The data we will be using for training our model is provided by Kaggle. Feel free to make the model better on your own and submit it to the Kaggle Titanic competition!
#If you get stuck during this project or would like to see an experienced developer copmplete it, check out the project walkthrough video which can be found in the “get help” menu in the bottom-right of this window.

In [2]:
import seaborn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
#1. The file passengers.csv contains the data of 892 passengers onboard the Titanic when it sank that fateful day.
#Let’s begin by loading the data into a pandas DataFrame named passengers. 
#Print passengers and inspect the columns. What features could we use to predict survival?

In [4]:
passengers = pd.read_csv('passengers.csv', delimiter =',')
#print(passengers)

In [5]:
#2. Given the saying, “women and children first,” Sex and Age seem like good features to predict survival. 
#Let’s map the text values in the Sex column to a numerical value. 
#Update Sex such that all values female are replaced with 1 and all values male are replaced with 0. Turn column into integers.

In [6]:
passengers['Sex']=passengers['Sex'].map({'male':0, 'female': 1})
print(passengers)

     PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name  Sex   Age  SibSp  \
0                              Braund, Mr. Owen Harris    0  22.0      1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...    1  38.0      1   
2                               Heikkinen, Miss. Laina    1  26.0      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)    1  35.0      1   
4                             Allen, Mr. William Henry    0  35.0      0   
..                                                 ...  ...   ...    ...   
886        

In [7]:
#3. Let’s take a look at Age. Print passengers['Age'].values. 
#You can see we have multiple missing values, or nans. Fill all the empty Age values in passengers with the mean age.
# inplace will modify passengers.Age, so it does not create a new column.
# # use the round function to round the mean value.

In [8]:
passengers['Age'].fillna(inplace=True, value=round(passengers['Age'].mean()))
print(passengers)

     PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name  Sex   Age  SibSp  \
0                              Braund, Mr. Owen Harris    0  22.0      1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...    1  38.0      1   
2                               Heikkinen, Miss. Laina    1  26.0      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)    1  35.0      1   
4                             Allen, Mr. William Henry    0  35.0      0   
..                                                 ...  ...   ...    ...   
886        

In [9]:
#4 Given the strict class system onboard the Titanic, let’s utilize the Pclass column, or the passenger class, as another feature. 
#Create a new column named FirstClass that stores 1 for all passengers in first class and 0 for all other passengers.
#Use apply(), which will apply a specific lambda function to every single value inside of PClass.
#We can then use that to map each PClass value to a new value in the FirstClass column.

In [10]:
passengers['FirstClass']=passengers['Pclass'].apply(lambda p: 1 if p ==1 else 0)
print(passengers) ##now only FirstClass passengers have a 1 in their column.

     PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name  Sex   Age  SibSp  \
0                              Braund, Mr. Owen Harris    0  22.0      1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...    1  38.0      1   
2                               Heikkinen, Miss. Laina    1  26.0      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)    1  35.0      1   
4                             Allen, Mr. William Henry    0  35.0      0   
..                                                 ...  ...   ...    ...   
886        

In [11]:
#5 Create a new column named SecondClass that stores 1 for all passengers in second class and 0 for all other passengers.
#Print passengers and inspect the DataFrame to ensure all the updates have been made.

In [12]:
passengers['SecondClass']=passengers['Pclass'].apply(lambda p: 1 if p ==2 else 0)
print(passengers)

     PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name  Sex   Age  SibSp  \
0                              Braund, Mr. Owen Harris    0  22.0      1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...    1  38.0      1   
2                               Heikkinen, Miss. Laina    1  26.0      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)    1  35.0      1   
4                             Allen, Mr. William Henry    0  35.0      0   
..                                                 ...  ...   ...    ...   
886        

In [13]:
#6 Now that we have cleaned our data, let’s select the columns we want to build our model on. 
# Select columns Sex, Age, FirstClass, and SecondClass and store them in a variable named features. Select column Survived and store it a variable named survival.
##Create an array

In [14]:
features=passengers[['Sex', 'Age', 'FirstClass', 'SecondClass']]
survived=passengers['Survived']

In [15]:
#7 Split the data into training and test sets using sklearn‘s train_test_split() method. 
# We’ll use the training set to train the model and the test set to evaluate the model.

In [16]:
train_features, test_features, train_labels, test_labels = train_test_split(features, survived)

In [17]:
##Normalize the Data
#8 Since sklearn‘s Logistic Regression implementation uses Regularization, we need to scale our feature data. 
# Create a StandardScaler object, .fit_transform() it on the training features, and .transform() the test features.

In [18]:
scaler=StandardScaler()
train_features = scaler.fit_transform(train_features)

test_features = scaler.transform(test_features)

In [19]:
#9 Create a LogisticRegression model with sklearn and .fit() it on the training data.
# Fitting the model will perform gradient descent to find the feature coefficients that minimize the log-loss for the training data.

In [20]:
model = LogisticRegression()
model.fit(train_features, train_labels)
##this will train our model to recognize which type of passengers survived and which
#-type of passengers didn't survive based on the features (Sex, Age, FirstClass, SecondClass)

LogisticRegression()

In [21]:
#10 .score() the model on the training data and print the training score.
# Scoring the model on the training data will run the data through the model and make final classifications on survival for each passenger in the training set. 
# The score returned is the percentage of correct classifications, or the accuracy.

In [22]:
print(model.score(train_features, train_labels))  ##result: about an 80% accuracy rate

0.781437125748503


In [23]:
#11 .score() the model on the test data and print the test score.
#Smilarly, scoring the model on the testing data will run the data through the model and make final classifications on survival for each passenger in the test set.
# w well did your model perform?

In [24]:
print(model.score(test_features, test_labels))  #result is almost the same as the train score

0.8295964125560538


In [25]:
#12 Print the feature coefficients determined by the model. 
# Which feature is most important in predicting survival on the sinking of the Titanic?

##Feature coefficients =Sex, Age, FirstClass, SecondClass. How significant were each of these feature coefficients in determining if someone survived.

In [26]:
print(model.score(test_features, test_labels))
print(model.coef_)  #the printed array results are in this order: Sex, Age, FirstClass, SecondClass. 

0.8295964125560538
[[ 1.14041457 -0.33208631  0.91372091  0.52413877]]


In [27]:
#13.Let’s use our model to make predictions on the survival of a few fateful passengers. Provided in the code editor is information for 3rd class passenger Jack and 1st class passenger Rose, stored in NumPy arrays. The arrays store 4 feature values, in the following order:
#Sex, represented by a 0 for male and 1 for female
#Age, represented as an integer in years
#FirstClass, with a 1 indicating the passenger is in first class
#SecondClass, with a 1 indicating the passenger is in second class
#A third array, You, is also provided in the code editor with empty feature values. Uncomment the line containing You and update the array with your information, or the information for some fictitious passenger. Make sure to enter all values as floats with a .!

In [28]:
Jack= np.array([0.0,20.0,0.0,0.0])
Rose= np.array([1.0,17.0,1.0,0.0])
You= np.array([0.0,47.0,0.0,1.0])

In [29]:
#14 Combine Jack, Rose, and You into a single NumPy array named sample_passengers

In [30]:
sample_passengers=np.array([Jack, Rose, You])

In [31]:
#15 Since our Logistic Regression model was trained on scaled feature data, we must also scale the feature data we are making predictions on. 
#Using the StandardScaler object created earlier, apply its .transform() method to sample_passengers and save the result to sample_passengers.
# Print sample_passengers to view the scaled features.

In [32]:
sample_passengers=scaler.transform(sample_passengers) ##normalize

In [33]:
#16 Who will survive, and who will sink? Use your model’s .predict() method on sample_passengers and print the result to find out.
# Want to see the probabilities that led to these predictions? Call your model’s .predict_proba() method on sample_passengers and print the result. 
#The 1st column is the probability of a passenger perishing on the Titanic, and the 2nd column is the probability of a passenger surviving the sinking (which was calculated by our model to make the final classification decision).

In [34]:
print(model.predict(sample_passengers))

print(model.predict_proba(sample_passengers))

[0 1 0]
[[0.88625317 0.11374683]
 [0.07425643 0.92574357]
 [0.80789486 0.19210514]]


In [35]:
# predict_proba Results: 
# [[0.88903321 0.11096679] Jack had an 88% chance of dying and an 11% chance of surviving
# [0.05447605 0.94552395]  Rose had a 5% chance of dying and a 94% chance of surviving
# [0.14235909 0.85764091]] I had a 14% chance of dying and a 85% of dying

In [1]:
print("hello")


hello


In [2]:
print("hi")

hi


In [3]:
print("ok")

ok


In [5]:
print("I hate covid")

I hate covid


In [1]:
print("we really have covid")

we really have covid


In [2]:
print(1+1)

2
